In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

Using TensorFlow backend.


In [0]:
#Read midi File 
mid = MidiFile('/content/drive/My Drive/MusicGeneration/Nintendo_-_Pokemon_Fire_Red_Route_1_Piano_Cover_Hard_Version.mid')

In [0]:
notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

In [0]:
notes = [list(note) for note in notes]

X = []
y = []
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)

In [0]:
model.fit(np.array(X), np.array(y), 32, 10, verbose=1)

Epoch 1/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0520
Epoch 2/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0471
Epoch 3/10
12770/12770 [==============================] - 71s 6ms/step - loss: 0.0451
Epoch 4/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0417
Epoch 5/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0345
Epoch 6/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0276
Epoch 7/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0242
Epoch 8/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0219
Epoch 9/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0204
Epoch 10/10
12770/12770 [==============================] - 72s 6ms/step - loss: 0.0175


In [0]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [0]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    #147 --> note_on
    #67 -> velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

In [0]:
from google.colab import files
files.download('LSTM_music.mid')